- Trees basics: node, leaf
- Entropy and gini
- Classification with trees
- Limits of trees

Missing:

- Decision tree implementation for classification, with decision function and display of the tree
- decision tree for regression with short example of implementation


# Decision Trees

---

---

Today we will explore a new **classification** machine learning algorithm (supervised for categorical data) called **Decision Trees**. 🌳

It is:
- one of the oldest ML algorithm
- extremely robust
- quite intuitive

We will also discover an extension of the Decision Trees: the **Random Forests** 🌳🌳🌳

Let's dig in!

---

# I. Trees Intuition

## I.1. Trees and linearly separable data

Trees can be efficient on non linearly separable data.

Let's have a toy example: what would be the conditions to sell a lot of ice creams on the beach?

- Warm temperature
- Sunny weather

We can draw this dataset on a simple scatter plot:

![](images/ice_cream_data.png)

Would you say this data is linearly separable? And why?

Though not linearly separable, this data can be well classified with a sequence of two questions:
- is the temperature warm?
- is the weather sunny?

Another way to represent it is the following:

![](images/ice_cream_tree.png)

## I.2. Vocabulary

On this tree, we can see three different objects:
- the **root node** is the first node, here `warm?`
- a **decision node** is just any node with two (or sometimes more) branches
- a **leaf** is does not contain any split, it contains a final prediction

## I.3. An example of more complex tree

<img src="https://drive.google.com/uc?export=view&id=1am_borVWERNA0BvOgsIy2HPqwJexAdD7">

---

# II. Decision trees for classification

## II.1. Greedy algorithm

Our tree is based on a so called **greedy algorithm**: meaning it only tries to optimize one step at a time.

In our case, it means our tree is not optimized globally, but one node at a time.

This can be seen as a recursive algorithm:
1. take all samples in a node
2. find a threshold in a feature that minimizes the disorder
3. split this into two new nodes
4. go back to 1. until your node is pure, thus becoming a leaf

The only question here is: how do we measure disorder?

## II.2. Disorder measurement

There are several ways to measure disorder. We will here cover two of them, the most used in decision trees:
- Entropy
- Gini impurity

### Entropy

One of the most common ways of measuring disorder is using **entropy**.

Entropy is a generic word, used in various fields like physics (thermodynamics) or computer science.

It always refers to a sort of disorder, and can be expressed as follow:
$$
E = -\sum p_i log_2 p_i
$$

Where $p_i$ is defined as the proportion of element $i$ in a subset.

Let's consider the following example:

![](images/root_node.png)

The entropy would be:
$$
E = - \frac{3}{10} log_2 \frac{3}{10} - \frac{7}{10} log_2 \frac{7}{10} \simeq 0.88
$$

Because we have:
- $p_{blue} = \frac{3}{10}$, since we have 3 blue balls out of 10
- $p_{red} = \frac{7}{10}$, since we have 7 blue balls out of 10

This entropy has several interesting properties for disorder measurement:
- if $p_{blue} = 0$, then $p_{red} = 1$ and $ E = 0 $
- if $p_{blue} = p_{red} = 0.5$, then $ E = 1$

![](images/Entropy.png)

### Gini impurity

Gini impurity is also a disorder measurement. The formula is the following:
$$
G(f)=\sum f_{i}(1-f_{i})=\sum (f_{i}-{f_{i}}^{2})=\sum f_{i}-\sum {f_{i}}^{2}=1-\sum {f_{i}}^{{2}}
$$

Where this time the proportion of element $i$ is defined by $f_i$.

On our same example:

![](images/root_node.png)

The gini impurity would be:
$$
G = 1 - \left(\frac{3}{10}\right)^2 - \left(\frac{7}{10}\right)^2 = 0.42
$$

In [ ]:
1-0.5*0.5-0.5*0.5

Once again we have our extreme cases:
- if $f_{blue} = 0$, then $f_{red} = 1$ and $ E = 0 $
- if $f_{blue} = f_{red} = 0.5$, then $ E = 0.5$

## II.3. Loss function

Finally, in order to split a node optimally, we need to compute the a loss to minimize is the measure of the **disorder decrease** thanks to our split.

To compute this loss, we consider the two children nodes, that we will call *left* and *right*.

The formula is the following:
$$
\mathcal{L} = \frac{m_{left}}{m} G_{left} + \frac{m_{right}}{m}G_{right}
$$

Where:
- $m$, $m_{left}$ and $m_{right}$ are the number of samples in each nodes respectively
- $G_{left}$ and $G_{right}$ are the *Gini impurities* of *left* and *right* nodes

> Of course, one can use entropy instead of gini impurity

Imagine we choose a split decision, we then have a **parent node** and **two children nodes**:
![](images/split_nodes.png)

Thus here the loss $\mathcal{L}$ would be:
$$
\mathcal{L} = \frac{3}{10}0.44 + \frac{7}{10} 0.41 = 0.42
$$

The algorithm will find the split that minimizes this entropy to create the new nodes.

Then, until we reach a limit (pure leaves or any given constraint), it will recursively keep splitting the data.

## II.4. Implementation

Decision trees are available in scikit-learn, with the following signature:
```python

class sklearn.tree.DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort='deprecated', ccp_alpha=0.0)

```

In [1]:
1 - (2/7)**2 - (5/7)**2

0.40816326530612246

In [2]:
1 - (1/3)**2 - (2/3)**2

0.4444444444444444

In [3]:
0.3*(1 - (1/3)**2 - (2/3)**2) + 0.7*(1 - (2/7)**2 - (5/7)**2)

0.419047619047619

Finally, given the "measure of disorder" (Entropy or Gini Impurity) of a parent dataset and of children datasets, we can compute the **Information Gain** which corresponds of the  **measure of the decrease** of our "measure of disorder"  after the data split.

As its name indicates, it represents the increase of knowledge when performing the split.

But we actually prefer to use the **Weighted Information Gain**, which takes into account the size of the subsets created.

If the data before the split contained 20 items and one of the resulting splits contained 2 items, then the weighted impurity of that subset would be 2/20 * impurity.

By doing so we are lowering the importance of the impurity of sets with few elements.

<p align="center">
<img src="https://drive.google.com/uc?export=view&id=1OTY2LCK_ZF9gogmxH9Bqmn4ner_3VlZE" width="500">
</p>

We have reached the **base case** (or **leaf node** 🍂) when **splitting on every feature results in the largest information gain**.

> 📚 **Resources**: 
- [What is Entropy and Information Gain](https://stackoverflow.com/questions/1859554/what-is-entropy-and-information-gain)
- [A Simple Explanation of Entropy in Decision Trees](https://bricaud.github.io/personal-blog/entropy-in-decision-trees/)

<p align="center">
<img src="https://drive.google.com/uc?export=view&id=1vGvHB-oPksgqCEI78jSUPwNU9d2AGV4v" width="500">
</p>

In [ ]:
import numpy as np

0.3*np.log2(0.3) + 0.7*np.log2(0.7)

### III.2.A. Definition

The **Entropy** is a **measure of randomness (or unpredictability)** in the dataset.

We define the **Entropy of a dataset** like this: 
![](https://chart.apis.google.com/chart?cht=tx&chl=E%3D-%5Csum_ip_i%5Clog_2p_i%20%2C)

where $p_i$ corresponds to the ratios of elements of each label `i` in the set

If you chart the **Binary Entropy function** (the Entropy for a random variable `X` that can take one out of 2 values (`a` and `b` for example) using the formula above for every possible value of `p(X=a)`, it looks like this: 

<p align="center">
<img src="https://i.stack.imgur.com/OUgcx.png">
</p>

> 🔦 **Hint**: It reaches its maximum when the probability is p=1/2, meaning that $p(X=a)=0.5$ or similarly $p(X=b)=0.5$ having a 50%/50% chance of being either a or b (uncertainty is at a maximum). The entropy function is at zero minimum when probability is p=1 or p=0 with complete certainty ( $p(X=a)=1$ or $p(X=a)=0$ respectively, latter implies $p(X=b)=1$ ).

### III.2.B. Illustration

<img src="https://drive.google.com/uc?export=view&id=1mGOOvmwCiBTxZCMeJ2h6ja14Frh1fM0c">

At each step, we can compute the Entropy of our splitted dataset by computing the **weighted mean of Entropy** of all subsets:

![](https://chart.apis.google.com/chart?cht=tx&chl=E_%7B%5Crm%20split%7D%3D%5Cfrac%7BN_1%7D%7BN%7DE_1%2B%5Cfrac%7BN_2%7D%7BN%7DE_2)

Splitting our dataset into two subsets "with less randomness" reduces our entropy.

## III.2. Gini Impurity

*Should you use Gini or Entropy?*
- Your performance will almost never change whether you use Gini impurity or Entropy.
- Entropy might be a little slower to compute

# II. Implementation with scikit-learn

## II.1. Coding a decision tree

In [ ]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)
X_small = X[:,:2] # We keep only 2 features for visualisation purposes

In [ ]:
print("X_small.shape={}".format(X_small.shape))
print("y.shape={}".format(y.shape))

In [ ]:
import matplotlib
from matplotlib import pyplot as plt

# Create figure to draw chart
plt.figure(2, figsize=(10, 8))

# Plot the training points
plt.scatter(X_small[:, 0], X_small[:, 1], c=y, cmap=plt.cm.Set1, edgecolor='k')

# Format chart
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')
plt.xticks(())
plt.yticks(())

plt.show()

We start by importing the proper method `DecisionTreeClassifier` from `sklearn` and to instanciate our Decision Tree classifier.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier() # Per default, criterion="gini"; you could specify criterion="entropy"

Then we fit our model as usual

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_small, y, random_state=0)

In [ ]:
dt.fit(X_train, y_train)

Finally we can retrieve predictions based on our model

In [ ]:
y_pred = dt.predict(X_test)

And retrieve a score (accuracy) of our model 

In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_pred)

print("Model has an accuracy of {}%".format(acc*100))

We can get a better understanding of our model by plotting the decision boundaries

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# Create figure to draw chart
plt.figure(2, figsize=(10, 8))

# We create a grid of points contained within [x_min, x_max]x[y_min, y_max] with step h=0.02
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
h = .02  # step size of the grid
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Retrieve predictions for each point of the grid
Z_dt = dt.predict(np.c_[xx.ravel(), yy.ravel()])
Z_dt = Z_dt.reshape(xx.shape)

# Plot the decision boundary (label predicted assigned to a color)
plt.pcolormesh(xx, yy, Z_dt, cmap=plt.cm.Paired)

# Plot also the training points
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, edgecolors='k', cmap=plt.cm.Paired)

# Format chart
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')
plt.xticks(())
plt.yticks(())
plt.show()

## II.2. Visualizing our tree

`sklearn` even allows us to visualize the tree. By exporting it to graphviz format and opening it with another library called `graphviz` (you will need to install it by running `pip install graphviz`).

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

# We load iris data again to retrieve features and classes names
iris = load_iris()

# We export the tree in graphviz format 
dot_data = export_graphviz(dt,
                           out_file=None,
                           feature_names=iris.feature_names[:2],  
                           class_names=iris.target_names,  
                           filled=True, rounded=True)

# We load the tree again with graphviz library in order to display it
graph = graphviz.Source(dot_data)
graph

## II.3. Interpretability

With trees, it becomes very easy to inspect and understand important features in our model decision. 

You just have to inspect `feature_importances_` attribute! 

In [ ]:
print(dt.feature_importances_)

Or if you want to make it more visual 🙂

In [ ]:
# Feature importances
features = iris.feature_names[:2]
importances = dt.feature_importances_
indices = np.argsort(importances)[::-1]
num_features = len(importances)

# Plot the feature importances of the tree
plt.figure()
plt.title("Feature importances")
plt.bar(range(num_features), importances[indices], color="g", align="center")
plt.xticks(range(num_features), [features[i] for i in indices], rotation='45')
plt.xlim([-1, num_features])
plt.show()

# Print values
for i in indices:
    print ("{0} - {1:.3f}".format(features[i], importances[i]))

## II.4. Impact and meaning of parameters

- **criterion**: The function to measure the quality of a split. For example `Gini Impurity` or `Entropy` 


- **max_depth**: The maximum depth of the tree


- **min_samples_split**: The minimum number of samples required to split an internal node


- **min_samples_leaf**: The minimum number of samples required to be at a leaf node.

 
- **min_weight_fraction_leaf** : The minimum weighted fraction of the sum total of weights (of all
    the input samples) required to be at a leaf node. Samples have
    equal weight when sample_weight is not provided.
    

- **max_features**: The number of features to consider when looking for the best split.


    Note: the search for a split does not stop until at least one
    valid partition of the node samples is found, even if it requires to
    effectively inspect more than ``max_features`` features.
    

- **max_leaf_nodes** : Grow a tree with ``max_leaf_nodes`` in best-first fashion.
    Best nodes are defined as relative reduction in impurity.
    If None then unlimited number of leaf nodes.
    

- **min_impurity_decrease**: A node will be split if this split induces a decrease of the impurity
    greater than or equal to this value.

---